In [1]:
from subdivision_learning.analysis.tree_visualizer.tree_visualizer import TreeVisualizer, InteractiveTreeLoader
%load_ext autoreload
%autoreload 2

Logs will be saved to: /home/jzxr91/projects/uc_workspace/ultracruise/logs/AV_Log_ipykernel_launcher.log


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [10]:
import copy
import functools
import time
from typing import Tuple

from subdivision_learning.analysis.visualization.tree_visualization import TreeVisualizerOpenCV
from subdivision_learning.benchmark.benchmark import Benchmark
from subdivision_learning.benchmark.utils.planner_eval_sample_viewer import visualize_planned_planner_eval_sample
from subdivision_learning.data_generator.sample_generators.planner_evalution_sample import PlannerEvaluationSample
from subdivision_learning.utils.planner_loader import get_default_search_based_planner
from subdivision_planner.src.common.config import Config
from subdivision_planner.src.mdp.experience import MDPExperience
from subdivision_planner.src.mdp.state import MDPContext, MDPState
from subdivision_planner.src.mdp.werling.concepts.free_space_grid_obstacles import OccupancyGridObstaclesActionValidator
from subdivision_planner.src.planners.search_based_planner import SearchBasedPlanner
from subdivision_planner.src.utils.algo_timer import AlgoTimer
from subdivision_planner.src.utils.visualization.state_visualizer import StateVisualizer
from matplotlib import pyplot as plt
from subdivision_planner.src.data_structures.canonic_sequence import CanonicFrame

In [ ]:
# from subdivision_learning.analysis.data_layer.pubsub_recording_master_parser import PubSubRecordingMasterParser
# path = '/data2/recordings/usa_integ/2020_11_02/Scenario_1_trial_1_host_1_t_07_37_20'
# master_parser = PubSubRecordingMasterParser(base_recording_path=path)
# tree_loader = InteractiveTreeLoader(canonic_sequence_parser=master_parser.canonic_sequence_parser)

In [3]:
benchmark = Benchmark.load('/data2/benchmarks/free_space/free_space_cars_as_obstacles_batch/TwoObstacles_baseline_no_noise')
sim_res = benchmark.get_simulation_result('TwoActors_060a5220-bd41-11eb-8be3-0242ac110002')
cs = sim_res.planner_inputs[0].observation
env = sim_res.environment_states[0]
internal_state = sim_res.planner_inputs[0].planner_internal_state
exec_info = sim_res.planner_outputs[0].execution_info
iters = exec_info.s_Data.e_i_ShallowSearchIterations
Config().mcts.use_only_shallow = True
Config().mcts.shallow_search_time = 0.00000001
Config().mcts.shallow_search_min_iterations = iters
print('iters', iters)

action = sim_res.planner_outputs[0].action

iters 805


In [5]:

def alternative_get_result(self) -> Tuple[MDPExperience, float, int]:
    """
    This is an alternative implementation of the get result method that doesn't delete the root node of the searcher
    :return: best results, change best result to None so we don't think there are results waiting
    """
    exp, ret, iterations = self._best_experience, self._best_return, self._iterations_run
    self._best_experience, self._best_return, self._iterations_run = None, None, None

    return exp, ret, iterations

In [6]:
planner: SearchBasedPlanner = get_default_search_based_planner()
planner._shallow_searcher.get_result = functools.partial(alternative_get_result, self=planner._shallow_searcher)

planner.set_state(internal_state, cs.map)

previous_driving_plan = copy.copy(planner.driving_plan)

print("Running planner")
new_action = planner.plan(cs)
print("action", new_action)

    

root_node = planner._shallow_searcher._root_node
# exp, ret, iterations = planner._shallow_searcher.get_result()
exp = planner._shallow_searcher.find_best_experience(planner._shallow_searcher._root_node)[0]

print("actions")
for a in exp.actions:
    print(a._action_type)




Running planner
action None
actions


In [11]:

cf = CanonicFrame(cs, new_action, planner.get_state(), planner.get_execution_info())


In [12]:
tv = TreeVisualizer()
tv.update_search_result(cf, root_node, exp)

In [ ]:
# tv = TreeVisualizer()
# tree_loader.register_search_update_callback(tv.update_search_result)